<a href="https://colab.research.google.com/github/Kylehanna/Instacart-Market-Basket-Analysis/blob/main/Notebooks/03_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [178]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import os 
from statsmodels.graphics.mosaicplot import mosaic
color = sns.color_palette()

from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

#from library.sb_utils import save_file

import warnings
warnings.filterwarnings("ignore")

In [179]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [180]:
df = pd.read_csv('/content/drive/MyDrive/GROUND GAME AI/00 Capstone 2 - Instacart Market Basket Analysis (Customer Segmentation)/Instacart-Market-Basket-Analysis/data/instacartData_train.csv')

In [181]:
df.head()

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,0,1,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs,112108,train,4,4,10,9.0
1,1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs,112108,train,4,4,10,9.0
2,2,1,10246,3,0,Organic Celery Hearts,83,4,fresh vegetables,produce,112108,train,4,4,10,9.0
3,3,1,49683,4,0,Cucumber Kirby,83,4,fresh vegetables,produce,112108,train,4,4,10,9.0
4,4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods,112108,train,4,4,10,9.0


In [182]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 16 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   Unnamed: 0              1384617 non-null  int64  
 1   order_id                1384617 non-null  int64  
 2   product_id              1384617 non-null  int64  
 3   add_to_cart_order       1384617 non-null  int64  
 4   reordered               1384617 non-null  int64  
 5   product_name            1384617 non-null  object 
 6   aisle_id                1384617 non-null  int64  
 7   department_id           1384617 non-null  int64  
 8   aisle                   1384617 non-null  object 
 9   department              1384617 non-null  object 
 10  user_id                 1384617 non-null  int64  
 11  eval_set                1384617 non-null  object 
 12  order_number            1384617 non-null  int64  
 13  order_dow               1384617 non-null  int64  
 14  or

In [183]:
# df = df['order_dow'].astype(object)

In [184]:
df.shape

(1384617, 16)

## Feature Engineering

In [185]:
#Total number of orders placed by each users
users = df.groupby(by='user_id')['order_number'].aggregate('max').to_frame('u_num_of_orders').reset_index()

In [186]:
users.head()

,user_id,u_num_of_orders
0,1,11
1,2,15
2,5,5
3,7,21
4,8,4


In [187]:
# Average number of products bought by the user in each purchase.
#1. First getting the total number of products in each order.
total_prd_per_order = df.groupby(by=['user_id', 'order_id'])['product_id'].aggregate('count').to_frame('total_products_per_order').reset_index()
total_prd_per_order.head(10)

,user_id,order_id,total_products_per_order
0,1,1187899,11
1,2,1492625,31
2,5,2196797,9
3,7,525192,9
4,8,880375,18
5,9,1094988,22
6,10,1822501,4
7,13,1827621,5
8,14,2316178,11
9,17,2180313,6


In [188]:
#2. Getting the average number of products purchased by each user
avg_products = total_prd_per_order.groupby(by=['user_id'])['total_products_per_order'].mean().to_frame('u_avg_prd').reset_index()
avg_products.head()

,user_id,u_avg_prd
0,1,11.0
1,2,31.0
2,5,9.0
3,7,9.0
4,8,18.0


In [189]:
#deleting the total_prd_per_order dataframe
del total_prd_per_order

In [190]:
#dow of most orders placed by each user
from scipy import stats
dow = df.groupby(by='user_id')['order_dow'].agg(lambda x: stats.mode(x)[0]).to_frame('dow_most_orders_u').reset_index()
dow.head()

,user_id,dow_most_orders_u
0,1,4
1,2,1
2,5,0
3,7,2
4,8,1


In [191]:
#hour of day when most orders placed by each user
from scipy import stats
hod = df.groupby(by='user_id')['order_hour_of_day'].agg(lambda x: stats.mode(x)[0]).to_frame('hod_most_orders_u').reset_index()
hod.head()

,user_id,hod_most_orders_u
0,1,8
1,2,11
2,5,11
3,7,11
4,8,14


In [192]:
#merging the user created features.

#1. merging avg_products with users
users = users.merge(avg_products, on='user_id', how='left')
#deleting avg_products
del avg_products
users.head()

,user_id,u_num_of_orders,u_avg_prd
0,1,11,11.0
1,2,15,31.0
2,5,5,9.0
3,7,21,9.0
4,8,4,18.0


In [193]:
#2. merging dow with users.
users = users.merge(dow, on='user_id', how='left')
#deleting dow
del dow
users.head()

,user_id,u_num_of_orders,u_avg_prd,dow_most_orders_u
0,1,11,11.0,4
1,2,15,31.0,1
2,5,5,9.0,0
3,7,21,9.0,2
4,8,4,18.0,1


In [194]:
#number of times a product was purchased.
prd = df.groupby(by='product_id')['order_id'].agg('count').to_frame('prd_count_p').reset_index()
prd.head()

,product_id,prd_count_p
0,1,76
1,2,4
2,3,6
3,4,22
4,5,1


In [195]:
#products reorder ratio.
reorder_p = df.groupby(by='product_id')['reordered'].agg('mean').to_frame('p_reordered_ratio').reset_index()
reorder_p.head()

,product_id,p_reordered_ratio
0,1,0.644737
1,2,0.250000
2,3,1.000000
3,4,0.636364
4,5,1.000000


In [196]:
#merging the reorder_p with prd
prd = prd.merge(reorder_p, on='product_id', how='left')
#deleting reorder_p
del reorder_p
prd.head()

,product_id,prd_count_p,p_reordered_ratio
0,1,76,0.644737
1,2,4,0.250000
2,3,6,1.000000
3,4,22,0.636364
4,5,1,1.000000


In [197]:
#how many times has a user bought the same product?
uxp = df.groupby(by=['user_id', 'product_id'])['order_id'].agg('count').to_frame('uxp_times_bought').reset_index()
uxp.head()

,user_id,product_id,uxp_times_bought
0,1,196,1
1,1,10258,1
2,1,13032,1
3,1,25133,1
4,1,26088,1


In [198]:
#reorder ratio of the user for each product.
reorder_uxp = df.groupby(by=['user_id', 'product_id'])['reordered'].agg('mean').to_frame('uxp_reordered_ratio').reset_index()
reorder_uxp.head()

,user_id,product_id,uxp_reordered_ratio
0,1,196,1.0
1,1,10258,1.0
2,1,13032,1.0
3,1,25133,1.0
4,1,26088,1.0


In [199]:
#merging the two dataframes into one
uxp = uxp.merge(reorder_uxp, on=['user_id', 'product_id'], how='left')
#deleting reorder_uxp
del reorder_uxp
uxp.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio
0,1,196,1,1.0
1,1,10258,1,1.0
2,1,13032,1,1.0
3,1,25133,1,1.0
4,1,26088,1,1.0


In [200]:
#merging users df into uxp
#the dataset includes samples of all eval_set (prior/train/test) data.
data = uxp.merge(users, on='user_id', how='left')
data.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u
0,1,196,1,1.0,11,11.0,4
1,1,10258,1,1.0,11,11.0,4
2,1,13032,1,1.0,11,11.0,4
3,1,25133,1,1.0,11,11.0,4
4,1,26088,1,1.0,11,11.0,4


In [201]:
#merging products df into data
data = data.merge(prd, on='product_id', how='left')
data.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,prd_count_p,p_reordered_ratio
0,1,196,1,1.0,11,11.0,4,1507,0.810219
1,1,10258,1,1.0,11,11.0,4,104,0.750000
2,1,13032,1,1.0,11,11.0,4,153,0.758170
3,1,25133,1,1.0,11,11.0,4,186,0.747312
4,1,26088,1,1.0,11,11.0,4,136,0.566176


In [202]:
del [users, prd, uxp]

In [203]:
data.shape

(1384617, 9)

In [204]:
#keeping only the train and test from eval_set orders dataframe.
order_future = orders.loc[((orders.eval_set == 'train') | (orders.eval_set == 'test')), ['user_id', 'eval_set', 'order_id']]
order_future.head()

,user_id,eval_set,order_id
10,1,train,1187899
25,2,train,1492625
38,3,test,2774568
44,4,test,329954
49,5,train,2196797


In [205]:
order_future.eval_set.unique()

array(['train', 'test'], dtype=object)

In [206]:
#merging the order_future with the data.
data = data.merge(order_future, on='user_id', how='left')
data.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,prd_count_p,p_reordered_ratio,eval_set,order_id
0,1,196,1,1.0,11,11.0,4,1507,0.810219,train,1187899
1,1,10258,1,1.0,11,11.0,4,104,0.750000,train,1187899
2,1,13032,1,1.0,11,11.0,4,153,0.758170,train,1187899
3,1,25133,1,1.0,11,11.0,4,186,0.747312,train,1187899
4,1,26088,1,1.0,11,11.0,4,136,0.566176,train,1187899


In [207]:
datapath = '/content/drive/MyDrive/GROUND GAME AI/00 Capstone 2 - Instacart Market Basket Analysis (Customer Segmentation)/Instacart-Market-Basket-Analysis/raw data/orders.csv'

orders = pd.read_csv(datapath)

In [208]:
orders.eval_set.value_counts()

prior    3214874
train     131209
test       75000
Name: eval_set, dtype: int64

In [209]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [215]:
#preparing the train df.
#using the df['eval_set']=='test as the training data due to prior dataset size (32M.
data_train = data.loc[(orders.eval_set == 'train'), ['user_id', 'eval_set', 'order_id']]
data_train.head()

,user_id,eval_set,order_id
10,1,train,1187899
25,2,train,1492625
49,5,train,2196797
74,8,train,880375
78,9,train,1094988


In [216]:
data_test = orders.loc[((orders.eval_set == 'test')), ['user_id', 'eval_set', 'order_id']]
data_test.head()

,user_id,eval_set,order_id
38,3,test,2774568
44,4,test,329954
53,6,test,1528013
96,11,test,1376945
102,12,test,1356845


In [217]:
data_test.shape

(75000, 3)

In [224]:
#merging the information from the order_proucts_train df into the data_train.
data_train = data_train.merge(data[['product_id', 'order_id', 'reordered']], on=['product_id', 'order_id'], how='left')
data_train.head()

KeyError: ignored

In [ ]:
#filling the NAN values
data_train.reordered.fillna(0, inplace=True)

In [ ]:
#setting user_id and product_id as index.
data_train = data_train.set_index(['user_id', 'product_id'])

#deleting eval_set, order_id as they are not needed for training.
data_train.drop(['eval_set', 'order_id'], axis=1, inplace=True)

In [ ]:
data_train.head()

In [ ]:
#preparing the test dataset.
data_test = data[data.eval_set == 'test']
data_test.head()

In [ ]:
#deleting unwanted columns
data_test.drop(['eval_set', 'order_id'], axis=1, inplace=True)

In [ ]:
#setting user_id and product_id as index.
data_test = data_test.set_index(['user_id', 'product_id'])

In [ ]:
data_test.head()

In [ ]:
#deleting unwanted df
del [aisles, departments, order_products_prior, order_products_train, orders, order_future, data] #order_future, data

In [ ]:
#resetting index
data_train.reset_index(inplace=True)
data_test.reset_index(inplace=True)

In [ ]:
data_train.head()

In [ ]:
#merging the aisles and department ids to with the train and test data
data_train = data_train.merge(products[['product_id', 'aisle_id']], on='product_id', how='left')
data_test = data_test.merge(products[['product_id', 'aisle_id']], on='product_id', how='left')

In [ ]:
#department
data_train = data_train.merge(products[['product_id', 'department_id']], on='product_id', how='left')
data_test = data_test.merge(products[['product_id', 'department_id']], on='product_id', how='left')

In [ ]:
#setting user_id and product_id as index.
data_test = data_test.set_index(['user_id', 'product_id'])
#setting user_id and product_id as index.
data_train = data_train.set_index(['user_id', 'product_id'])

In [ ]:
y_train = data_train.reordered
X_train = data_train.drop(columns=['reordered'])

# Save preprocessed data in new csv

In [ ]:
'''# save the data to a new csv file
#datapath = '../../data'
#save_file(df, 'processed.csv', datapath)

df.to_csv('processed_data.csv',index=False)'''